In [1]:
import urllib2

In [2]:
class wordDictionary:
    def __init__(self, url):
        #Turn into dictionary of the form (word category, list of words within that category)
        data = urllib2.urlopen(url)
        self.dictionary = {} 
        for line in data:
            words = line.split(":")
            category = words[0]
            wordList = words[1].split(", ")
            wordList[0] = wordList[0].replace('\t','')
            wordList[-1] = wordList[-1].replace('\r','').replace('\n','')
            self.dictionary[words[0]] = wordList
    
    def getLetters(self, category, position, firstLetter = None, secondLetter = None, thirdLetter = None):
        letters = []
        for word in self.dictionary[category]:
            if ((word[0] == firstLetter or firstLetter == None) and (word[1] == secondLetter or secondLetter == None) and \
                (word[2] == thirdLetter or thirdLetter == None)):
                newLetter = word[position]
                if not newLetter in letters: 
                    letters.append(newLetter)
        return letters
            
    def getWords(self, category = None, firstLetter = None, secondLetter = None, thirdLetter = None):
        words = []
        for word in self.dictionary[category]:
            if ((word[0] == firstLetter or firstLetter == None) and (word[1] == secondLetter or secondLetter == None) and \
                (word[2] == thirdLetter or thirdLetter == None)):
                words.append(word)
        return words
    
def intersection(L1, L2):
    newL = []
    for e in L1:
        if e in L2:
            newL.append(e)
    return newL
            
        
                
dictionary = wordDictionary("http://slazebni.cs.illinois.edu/fall15/assignment2/wordlist.txt")
print intersection(dictionary.getLetters("noun", 2, firstLetter = 'H'),dictionary.getLetters("noun", 2, firstLetter = "L"))
print dictionary.getWords("noun", firstLetter = "H", thirdLetter = "T")#firstLetter = "H")

['T', 'D', 'P', 'G']
['HAT', 'HIT']


In [3]:
class LetterCSP:
    def __init__(self, url, dictionary):
        data = urllib2.urlopen(url)
        i = 0
        self.valTracker = {}
        for line in data:
            if i == 0:
                self.size = int(line)
                for j in range(1,self.size+1):
                    self.valTracker[j] = []
            else:
                labels = line.split(":")
                category = labels[0]
                entries = labels[1].split(",")
                entries[-1] = entries[-1].replace("\r\n","")
                position = 0
                for entry in entries:
                    key = int(entry)
                    if not self.valTracker[key]: 
                        self.valTracker[key] = dictionary.getLetters(category, position)
                    else:
                        self.valTracker[key] = intersection(self.valTracker[key], dictionary.getLetters(category, position))
                    position += 1
                    
                #list contraints
            i += 1
            
        

CSP = LetterCSP("http://slazebni.cs.illinois.edu/fall15/assignment2/puzzle1.txt", dictionary)
print CSP.valTracker

{1: ['A', 'N', 'O', 'P', 'Q', 'S', 'Y'], 2: ['A', 'B', 'C', 'D', 'F', 'H', 'I', 'L', 'M', 'N', 'O', 'R', 'S', 'W'], 3: ['A', 'E', 'L', 'R', 'T'], 4: ['A', 'M', 'S', 'W'], 5: ['O', 'A'], 6: ['H', 'K', 'A', 'E', 'M', 'O', 'W', 'Y', 'N', 'T', 'F', 'P', 'Z'], 7: ['E', 'D'], 8: ['R', 'A', 'Y', 'I', 'E', 'O'], 9: ['E', 'W']}


In [4]:
class CSPSolver:
    def __init__(self, csp):
        self.csp = csp
        
    def backtrackSearch(self):
        assignment = [None for i in range(self.csp.size)]
        valTracker = self.csp.valTracker
        return self.recursiveBacktrack(assignment, valTracker)
    
    def recursiveBacktrack(self, assignment, valTracker):
        if self.isComplete(assignment):
            return (assignment[:],)
        
        solutions = ()
        key = self.pickVariable(assignment, valTracker)
        values = self.orderValues(valTracker[key])
        #newValTracker = valTracker.copy()
        for value in values:
            if self.checkConsistency(key, value, valTracker):
                assignment[key-1] = value
                result = self.recursiveBacktrack(assignment, valTracker)
                if result != None:
                    solutions = solutions + result
                assignment[key-1] = None
        if solutions == ():
            return None
        else:
            return solutions
                    
    def pickVariable(self, assignment, valTracker):
        minNumConstraint = None
        minConstrainedVariable = None
        for key in valTracker:
            if assignment[key-1] == None:
                numConstraint = len(valTracker[key])
                if minNumConstraint == None:
                    minNumConstraint = numConstraint
                    minConstrainedVariable = key
                if numConstraint < minNumConstraint:
                    minNumConstraint = numConstraint
                    minConstrainedVariable = key
        return minConstrainedVariable
    
    def isComplete(self, assignment):
        for key in assignment:
            if key == None:
                return False
        return True
        
    def orderValues(self, List):
        return List[:]
    
    def checkConsistency(self, key, value, valTracker):
        return True

def backtrackSearch(valTracker):
        assignment = [None for i in range(len(valTracker))]
        return recursiveBacktrack(assignment, valTracker)    
    
def recursiveBacktrack(assignment, valTracker):
        if isComplete(assignment):
            #print assignment
            return (assignment[:],)
        
        solutions = ()
        key = pickVariable(assignment, valTracker)
        values = orderValues(valTracker[key])
        #newValTracker = valTracker.copy()
        for value in values:
            if checkConsistency(key, value, valTracker):
                assignment[key-1] = value
                result = recursiveBacktrack(assignment, valTracker)
                #print result
                if result != None:
                    solutions = solutions + result
                assignment[key-1] = None
        print solutions
        if solutions == ():
            return None
        else:
            return solutions

def pickVariable(assignment, valTracker):
        minNumConstraint = None
        minConstrainedVariable = None
        for key in valTracker:
            if assignment[key-1] == None:
                numConstraint = len(valTracker[key])
                if minNumConstraint == None:
                    minNumConstraint = numConstraint
                    minConstrainedVariable = key
                if numConstraint < minNumConstraint:
                    minNumConstraint = numConstraint
                    minConstrainedVariable = key
        return minConstrainedVariable
    
def isComplete(assignment):
    for key in assignment:
        if key == None:
            return False
    return True
        
def orderValues(List):
    return List[:]
    
def checkConsistency(key, value, valTracker):
    return True

valTracker = {1: ["A", "B", "C"], 2 : ["D", "E", "F"]}
print backtrackSearch(valTracker)
#letterSolver = CSPSolver(CSP)
#solution = letterSolver.backtrackSearch()

(['A', 'D'], ['A', 'E'], ['A', 'F'])
(['B', 'D'], ['B', 'E'], ['B', 'F'])
(['C', 'D'], ['C', 'E'], ['C', 'F'])
(['A', 'D'], ['A', 'E'], ['A', 'F'], ['B', 'D'], ['B', 'E'], ['B', 'F'], ['C', 'D'], ['C', 'E'], ['C', 'F'])
(['A', 'D'], ['A', 'E'], ['A', 'F'], ['B', 'D'], ['B', 'E'], ['B', 'F'], ['C', 'D'], ['C', 'E'], ['C', 'F'])
